In [7]:
import numpy as np
from typing import Union

class Frame:
    def __init__(self, r: np.ndarray, t: np.ndarray) -> None:
        """
        Initializes a Frame object with rotation and translation.

        Parameters:
        r (np.ndarray): Rotation matrix (3x3)
        t (np.ndarray): Translation vector (3,)
        """
        self.rotation = np.array(r)
        self.translation = np.array(t)

    def __array__(self):
        """
        Returns the Frame as a 4x4 homogeneous transformation matrix.
        """
        out = np.eye(4, dtype=np.float32)
        out[:3, :3] = self.rotation
        out[:3, 3] = self.translation
        return out

    def __str__(self) -> str:
        """
        String representation of the Frame as a numpy array.
        """
        return np.array_str(np.array(self), precision=4, suppress_small=True)

    def inv(self) -> 'Frame':
        """
        Returns the inverse of the Frame.

        Returns:
        Frame: The inverted Frame.
        """
        return Frame(self.rotation.T, -(self.rotation.T @ self.translation))

    def __matmul__(self, other: Union[np.ndarray, 'Frame']) -> 'Frame':
        """
        Matrix multiplication for Frame objects or with numpy arrays.
        
        Args:
        other (Union[np.ndarray, Frame]): Another Frame or a numpy ndarray.

        Returns:
        Frame: Resultant Frame from the matrix multiplication.
        """
        if isinstance(other, np.ndarray):
            # Multiply the rotation with the transposed other, then add translation
            return (self.rotation @ other.T).T + self.translation
        elif isinstance(other, Frame):
            # Multiply the rotation matrices and calculate the new translation
            return Frame(self.rotation @ other.rotation, self.rotation @ other.translation + self.translation)
        else:
            raise TypeError(f"Unsupported operand type(s) for @: 'Frame' and '{type(other).__name__}'")


In [8]:
import numpy as np

class Vector:
    def __init__(self, x: float, y: float, z: float) -> None:
        """
        Initializes a Vector object with x, y, z coordinates.

        Parameters:
        x (float): X coordinate
        y (float): Y coordinate
        z (float): Z coordinate
        """
        self.coords = np.array([x, y, z], dtype=np.float32)

    def __str__(self) -> str:
        """
        String representation of the Vector.
        """
        return f"Vector({self.coords[0]:.2f}, {self.coords[1]:.2f}, {self.coords[2]:.2f})"

    def __add__(self, other: 'Vector') -> 'Vector':
        """
        Adds two Vector objects.

        Args:
        other (Vector): Another Vector.

        Returns:
        Vector: Sum of the two vectors.
        """
        result = self.coords + other.coords
        return Vector(result[0], result[1], result[2])

    def __sub__(self, other: 'Vector') -> 'Vector':
        """
        Subtracts two Vector objects.

        Args:
        other (Vector): Another Vector.

        Returns:
        Vector: Difference of the two vectors.
        """
        result = self.coords - other.coords
        return Vector(result[0], result[1], result[2])

    def dot(self, other: 'Vector') -> float:
        """
        Computes the dot product of two Vector objects.

        Args:
        other (Vector): Another Vector.

        Returns:
        float: Dot product of the two vectors.
        """
        return np.dot(self.coords, other.coords)

    def cross(self, other: 'Vector') -> 'Vector':
        """
        Computes the cross product of two Vector objects.

        Args:
        other (Vector): Another Vector.

        Returns:
        Vector: Cross product of the two vectors.
        """
        result = np.cross(self.coords, other.coords)
        return Vector(result[0], result[1], result[2])

    def magnitude(self) -> float:
        """
        Computes the magnitude of the Vector.

        Returns:
        float: Magnitude of the vector.
        """
        return np.linalg.norm(self.coords)

# Example Usage
v1 = Vector(1, 2, 3)
v2 = Vector(4, 5, 6)

# Add vectors
v3 = v1 + v2
print(f"v1 + v2 = {v3}")

# Subtract vectors
v4 = v1 - v2
print(f"v1 - v2 = {v4}")

# Dot product
dot_product = v1.dot(v2)
print(f"v1 . v2 = {dot_product}")

# Cross product
v5 = v1.cross(v2)
print(f"v1 x v2 = {v5}")

# Magnitude
mag_v1 = v1.magnitude()
print(f"|v1| = {mag_v1}")


v1 + v2 = Vector(5.00, 7.00, 9.00)
v1 - v2 = Vector(-3.00, -3.00, -3.00)
v1 . v2 = 32.0
v1 x v2 = Vector(-3.00, 6.00, -3.00)
|v1| = 3.7416574954986572


In [9]:
def read_calbody_file(file_path: str):
    """
    Reads a CALBODY.TXT file and extracts the d_i, a_i, and c_i vectors into separate lists.

    Args:
    file_path (str): Path to the CALBODY.TXT file.

    Returns:
    tuple: Three lists containing d_i, a_i, and c_i vectors respectively.
    """
    d_vectors = []  # List to store the d_i vectors
    a_vectors = []  # List to store the a_i vectors
    c_vectors = []  # List to store the c_i vectors

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # First line contains N_D, N_A, N_C, and filename
        header = lines[0].split(',')
        N_D = int(header[0].strip())  # Number of d_i markers (base)
        N_A = int(header[1].strip())  # Number of a_i markers (calibration object)
        N_C = int(header[2].strip())  # Number of c_i markers (EM markers)

        # Extract d_i vectors from lines 2 to N_D + 1
        for i in range(1, N_D + 1):
            d_coords = list(map(float, lines[i].split(',')))
            d_vectors.append(Vector(d_coords[0], d_coords[1], d_coords[2]))

        # Extract a_i vectors from lines N_D + 1 to N_D + N_A + 1
        for i in range(N_D + 1, N_D + N_A + 1):
            a_coords = list(map(float, lines[i].split(',')))
            a_vectors.append(Vector(a_coords[0], a_coords[1], a_coords[2]))

        # Extract c_i vectors from lines N_D + N_A + 1 to N_D + N_A + N_C + 1
        for i in range(N_D + N_A + 1, N_D + N_A + N_C + 1):
            c_coords = list(map(float, lines[i].split(',')))
            c_vectors.append(Vector(c_coords[0], c_coords[1], c_coords[2]))

    return d_vectors, a_vectors, c_vectors, N_D, N_A, N_C

# Example Usage
file_path = "./PA_1_Data/pa1-debug-a-calreadings.txt"
d_vectors, a_vectors, c_vectors, N_D, N_A, N_C = read_calbody_file(file_path)

# Output the vectors
print("d_i Vectors (Base Markers):")
for vec in d_vectors:
    print(vec)
assert len(d_vectors) == N_D

print("\na_i Vectors (Calibration Object Markers):")
for vec in a_vectors:
    print(vec)
assert len(a_vectors) == N_A

print("\nc_i Vectors (EM Markers on Calibration Object):")
for vec in c_vectors:
    print(vec)
assert len(c_vectors) == N_C


d_i Vectors (Base Markers):
Vector(0.00, 0.00, -1500.00)
Vector(0.00, 0.00, -1350.00)
Vector(0.00, 150.00, -1500.00)
Vector(0.00, 150.00, -1350.00)
Vector(150.00, 0.00, -1500.00)
Vector(150.00, 0.00, -1350.00)
Vector(150.00, 150.00, -1500.00)
Vector(150.00, 150.00, -1350.00)

a_i Vectors (Calibration Object Markers):
Vector(209.30, 208.87, -1288.97)
Vector(205.19, 207.84, -1039.00)
Vector(211.50, 458.85, -1287.91)
Vector(207.39, 457.83, -1037.95)
Vector(459.26, 206.65, -1284.87)
Vector(455.15, 205.63, -1034.91)
Vector(461.45, 456.64, -1283.81)
Vector(457.35, 455.62, -1033.85)

c_i Vectors (EM Markers on Calibration Object):
Vector(209.30, 208.87, 211.03)
Vector(207.25, 208.35, 336.01)
Vector(205.19, 207.84, 461.00)
Vector(210.40, 333.86, 211.56)
Vector(208.35, 333.35, 336.54)
Vector(206.29, 332.84, 461.53)
Vector(211.50, 458.85, 212.09)
Vector(209.44, 458.34, 337.07)
Vector(207.39, 457.83, 462.05)
Vector(334.28, 207.76, 213.08)
Vector(332.23, 207.25, 338.06)
Vector(330.17, 206.74, 463.

In [10]:
def read_calreadings_file(file_path: str):
    """
    Reads a CALREADINGS.TXT file and organizes data by frame number. Each frame contains three lists of Vector objects
    for d_i, a_i, and c_i vectors.

    Args:
    file_path (str): Path to the CALREADINGS.TXT file.

    Returns:
    dict: A dictionary where the key is the frame number, and the value is a dictionary containing:
        - d_vectors: List of Vector objects for d_i coordinates
        - a_vectors: List of Vector objects for a_i coordinates
        - c_vectors: List of Vector objects for c_i coordinates
    """
    frames_data = {}  # Dictionary to store data for each frame

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # First line contains N_D, N_A, N_C, N_frames, and filename
        header = lines[0].split(',')
        N_D = int(header[0].strip())  # Number of d_i markers
        N_A = int(header[1].strip())  # Number of a_i markers
        N_C = int(header[2].strip())  # Number of c_i markers
        N_frames = int(header[3].strip())  # Number of data frames
        filename = header[4].strip()  # The filename is stored here as a string

        # Initialize the reading process
        line_index = 1  # Start reading after the header

        for frame_num in range(1, N_frames + 1):
            frame_dict = {
                'd_vectors': [],
                'a_vectors': [],
                'c_vectors': []
            }


            # Extract d_i vectors (N_D lines)
            for _ in range(N_D):
                d_coords = list(map(float, lines[line_index].split(',')))
                frame_dict['d_vectors'].append(Vector(d_coords[0], d_coords[1], d_coords[2]))
                line_index += 1

            # Extract a_i vectors (N_A lines)
            for _ in range(N_A):
                a_coords = list(map(float, lines[line_index].split(',')))
                frame_dict['a_vectors'].append(Vector(a_coords[0], a_coords[1], a_coords[2]))
                line_index += 1

            # Extract c_i vectors (N_C lines)
            for _ in range(N_C):
                c_coords = list(map(float, lines[line_index].split(',')))
                frame_dict['c_vectors'].append(Vector(c_coords[0], c_coords[1], c_coords[2]))
                line_index += 1

            # Store the frame data in the dictionary
            frames_data[frame_num] = frame_dict

    return frames_data

# Example Usage
file_path = "./PA_1_Data/pa1-debug-a-calreadings.txt"
frames_data = read_calreadings_file(file_path)

# Output the vectors for each frame
for frame_num, frame_data in frames_data.items():
    print(f"Frame {frame_num}:")

    print("  d_i Vectors (Base Markers):")
    for vec in frame_data['d_vectors']:
        print(f"    {vec}")

    print("  a_i Vectors (Calibration Object Markers):")
    for vec in frame_data['a_vectors']:
        print(f"    {vec}")

    print("  c_i Vectors (EM Markers on Calibration Object):")
    for vec in frame_data['c_vectors']:
        print(f"    {vec}")

    print()


Frame 1:
  d_i Vectors (Base Markers):
    Vector(0.00, 0.00, -1500.00)
    Vector(0.00, 0.00, -1350.00)
    Vector(0.00, 150.00, -1500.00)
    Vector(0.00, 150.00, -1350.00)
    Vector(150.00, 0.00, -1500.00)
    Vector(150.00, 0.00, -1350.00)
    Vector(150.00, 150.00, -1500.00)
    Vector(150.00, 150.00, -1350.00)
  a_i Vectors (Calibration Object Markers):
    Vector(209.30, 208.87, -1288.97)
    Vector(205.19, 207.84, -1039.00)
    Vector(211.50, 458.85, -1287.91)
    Vector(207.39, 457.83, -1037.95)
    Vector(459.26, 206.65, -1284.87)
    Vector(455.15, 205.63, -1034.91)
    Vector(461.45, 456.64, -1283.81)
    Vector(457.35, 455.62, -1033.85)
  c_i Vectors (EM Markers on Calibration Object):
    Vector(209.30, 208.87, 211.03)
    Vector(207.25, 208.35, 336.01)
    Vector(205.19, 207.84, 461.00)
    Vector(210.40, 333.86, 211.56)
    Vector(208.35, 333.35, 336.54)
    Vector(206.29, 332.84, 461.53)
    Vector(211.50, 458.85, 212.09)
    Vector(209.44, 458.34, 337.07)
    Vector(

In [11]:
import numpy as np
from scipy.optimize import least_squares
from scipy.spatial.transform import Rotation as R

def rodrigues_to_rotation_matrix(rvec):
    """Convert a Rodrigues vector to a 3x3 rotation matrix."""
    return R.from_rotvec(rvec).as_matrix()

def residuals(params, a_points, A_points):
    """Compute the residuals (differences) between transformed a_points and A_points."""
    rvec = params[:3]  # First three parameters are the axis-angle representation of rotation
    t = params[3:]  # Last three parameters are the translation vector

    # Compute rotation matrix from the axis-angle representation
    R_matrix = rodrigues_to_rotation_matrix(rvec)

    # Transform a_points using the current R and t
    transformed_a = np.dot(a_points, R_matrix.T) + t

    # Compute residuals (difference between transformed a_points and A_points)
    return (transformed_a - A_points).ravel()  # Flatten to a 1D array for least_squares

def point_cloud_registration_least_squares(A_points, a_points):
    """
    Perform point cloud registration between A_points and a_points using least squares.
    
    Args:
    A_points (np.ndarray): Nx3 array of points from calreadings.
    a_points (np.ndarray): Nx3 array of points from calbody.
    
    Returns:
    R (np.ndarray): 3x3 optimal rotation matrix.
    t (np.ndarray): 3x1 translation vector.
    """
    # Initial guess for parameters: rotation as [0, 0, 0] (no rotation), translation as [0, 0, 0]
    initial_params = np.zeros(6)

    # Use least squares to minimize the residuals
    result = least_squares(residuals, initial_params, args=(a_points, A_points))

    # Extract the optimal parameters from the result
    rvec_optimal = result.x[:3]
    t_optimal = result.x[3:]

    # Convert the optimal Rodrigues vector to a rotation matrix
    R_optimal = rodrigues_to_rotation_matrix(rvec_optimal)

    return R_optimal, t_optimal

def perform_registration_for_frames(calreadings_frames, calbody_a_vectors):
    """
    Perform point cloud registration for each frame in calreadings.
    
    Args:
    calreadings_frames (dict): Dictionary where each key is a frame number and value is a dict of A_i points.
    calbody_a_vectors (list): List of a_i vectors from calbody.
    
    Returns:
    dict: Dictionary with frame numbers as keys and (R, t) as values where R is the rotation matrix and t is the translation vector.
    """
    registration_results = {}

    # Convert calbody a_vectors to a numpy array
    a_points = np.array([vec.coords for vec in calbody_a_vectors])

    # Perform registration for each frame
    for frame_num, frame_data in calreadings_frames.items():
        A_points = np.array([vec.coords for vec in frame_data['a_vectors']])

        # Perform point cloud registration using least squares to find R and t
        R_optimal, t_optimal = point_cloud_registration_least_squares(A_points, a_points)

        # Store the result for this frame
        registration_results[frame_num] = {'rotation': R_optimal, 'translation': t_optimal}

    return registration_results

# Example Usage:
calreadings_frames = frames_data
calbody_a_vectors = a_vectors
# Perform registration for each frame
registration_results = perform_registration_for_frames(calreadings_frames, calbody_a_vectors)

# Output the registration results
for frame_num, result in registration_results.items():
    print(f"Frame {frame_num}:")
    print(f"  Rotation matrix (R):\n{result['rotation']}")
    print(f"  Translation vector (t): {result['translation']}")
    print()


Frame 1:
  Rotation matrix (R):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
  Translation vector (t): [0. 0. 0.]

Frame 2:
  Rotation matrix (R):
[[ 9.99316743e-01 -2.82571604e-02  2.38239439e-02]
 [ 2.82503422e-02  9.99600685e-01  6.22772499e-04]
 [-2.38320284e-02  5.06875820e-05  9.99715976e-01]]
  Translation vector (t): [ 36.1040255   -3.26179798 242.2408329 ]

Frame 3:
  Rotation matrix (R):
[[ 0.99939304 -0.03193952  0.01390744]
 [ 0.03173441  0.99938783  0.01472743]
 [-0.01436932 -0.01427715  0.99979482]]
  Translation vector (t): [ 26.65991937 255.36388633   5.18728448]

Frame 4:
  Rotation matrix (R):
[[ 9.99955839e-01  9.39736326e-03 -9.59956480e-05]
 [-9.38914352e-03  9.99416625e-01  3.28367790e-02]
 [ 4.04518787e-04 -3.28344276e-02  9.99460723e-01]]
  Translation vector (t): [ -2.00021586 286.08411721 245.24653407]

Frame 5:
  Rotation matrix (R):
[[ 0.99963609 -0.02534023  0.0092501 ]
 [ 0.02526592  0.99964824  0.00806414]
 [-0.00945119 -0.00782749  0.9999247 ]]
  Translation vec

In [12]:
import logging
from pathlib import Path
import numpy as np
import click
from rich.logging import RichHandler

# Configure logging
logging.basicConfig(
    level="DEBUG",
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)]
)

log = logging.getLogger(__name__)

@click.command()
@click.option("-d", "--data_dir", default="data", help="Where the data is.")
@click.option("-o", "--output_dir", default="outputs", help="Where the output is.")

def main(data_dir: str = "data", output_dir: str = "outputs") -> None:
    """
    Main function for setting up Frame examples and logging operations.
    
    Args:
    data_dir (str): Directory where the data is.
    output_dir (str): Directory where the output is.
    """
    data_dir = Path(data_dir).resolve()

    # Example rotation matrix and translation vectors
    rot = np.eye(3)
    trans = np.array([1.0, 0, 0])

    # Create Frame instances
    a = Frame(rot, trans)
    b = Frame(rot, np.array([0, 2, 0]))
    c = np.array([1.0, 0, 0])

    # Log the Frame objects and their operations
    log.debug(f"Example Frame A:\n{a}")
    log.debug(f"Example Frame B:\n{b}")
    
    # Perform multiplication between Frame A and B
    log.debug(f"Multiplication Example 1 (A @ B):\n{a @ b}")

    # Perform multiplication between Frame A and numpy array C
    log.debug(f"Multiplication Example 2 (A @ C):\n{a @ c}")

if __name__ == "__main__":
    main()


Usage: ipykernel_launcher.py [OPTIONS]
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --f


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
import setuptools

setuptools.setup(
    name="my_package",
    version="0.1",
    description="A sample Python package",
    author="John Doe",
    author_email="jdoe@example.com",
    packages=setuptools.find_packages(),
    install_requires=[
        'numpy',
        'scipy',
        'rich',
        'click',
    ],
)
